In [45]:
import numpy as np

def Skew(w):
    skew_mat = np.array([[0, -w[2], w[1]],
                        [w[2], 0, -w[0]],
                        [-w[1], w[0], 0]], dtype = 'object')
    return skew_mat

def matrix_exp(w_hat, theta):
    skew_w = Skew(w_hat)
    R = np.eye(3) + np.sin(theta) * skew_w + (1 - np.cos(theta)) * np.matmul(skew_w, skew_w)
    return R

def screw_exp(screw, theta):
    w_hat = screw[:3]
    v = screw[3:]

    w_hat_skew = Skew(w_hat)

    R = matrix_exp(w_hat, theta)
    G_theta = np.eye(3) * theta + (1 - np.cos(theta)) * w_hat_skew + (theta - np.sin(theta)) * np.matmul(w_hat_skew, w_hat_skew)

    T = np.vstack((np.hstack((R, np.matmul(G_theta, v.reshape((-1, 1))))),
                       [0, 0, 0, 1]))
    return T
    
def Ad(T):
    p = T[:3, 3]
    R = T[:3, :3]
    p_mat = Skew(p)
    Adjoint = np.vstack((np.hstack((R, np.zeros((3, 3)))),
                             np.hstack((p_mat @ R, R))))
    return Adjoint

In [46]:
import numpy as np
import time

from scipy.linalg import logm

w = np.array([[0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1]])
q = np.array([[0, 0, 0], [0, 0, 0.1253], [0, 0.0834, 0.1253], [0, 0.1668, 0.1253], [0, 0.1668, 0.0753]])
v = -np.cross(w,q)
s1 = np.vstack([w[0].reshape(-1,1), v[0].reshape(-1,1)])
s2 = np.vstack([w[1].reshape(-1,1), v[1].reshape(-1,1)])
s3 = np.vstack([w[2].reshape(-1,1), v[2].reshape(-1,1)])
s4 = np.vstack([w[3].reshape(-1,1), v[3].reshape(-1,1)])
s5 = np.vstack([w[4].reshape(-1,1), v[4].reshape(-1,1)])

q_end = np.array([[0], [0.1668], [0.0479]])
M = np.vstack([np.hstack([np.eye(3), q_end]),
                  np.array([0, 0, 0, 1])])

# Initial Pos = [0; 0.12; 0.0665]
th = np.deg2rad([[0.0315], [52.0915], [-86.5612], [34.4697], [-0.0315]])

pos_d = np.array([[0.017, 0.12, 0.0665], [0.035, 0.12, 0.0665], [0.055, 0.12, 0.0665],
                  [0.075, 0.12, 0.0665], [0.055, 0.12, 0.0665], [0.035, 0.12, 0.0665],
                  [0.017, 0.12, 0.0665], [0, 0.12, 0.0665], [-0.017, 0.12, 0.0665],
                  [-0.035, 0.12, 0.0665], [-0.055, 0.12, 0.0665], [-0.075, 0.12, 0.0665],
                  [-0.055, 0.12, 0.0665], [-0.035, 0.12, 0.0665], [-0.017, 0.12, 0.0665],
                  [0, 0.12, 0.0665]])

        # Forward Kinematic
exp1 = screw_exp(s1, th[0])
exp2 = screw_exp(s2, th[1])
exp3 = screw_exp(s3, th[2])
exp4 = screw_exp(s4, th[3])
exp5 = screw_exp(s5, th[4])
T_sb = exp1 @ exp2 @ exp3 @ exp4 @ exp5 @ M 
T_sb = T_sb.astype(dtype = 'float64')

        # Inverse Kinematic
T_bs = np.linalg.inv(T_sb)
AdT_bs = Ad(T_bs).astype(dtype = 'float64')

Js1 = s1
Js2 = Ad(exp1) @ s2
Js3 = Ad(exp1 @ exp2) @ s3
Js4 = Ad(exp1 @ exp2 @ exp3) @ s4
Js5 = Ad(exp1 @ exp2 @ exp3 @ exp4) @ s5
Js = np.hstack([Js1, Js2, Js3, Js4, Js5])
Js = Js.astype(dtype = 'float64')
Jb = AdT_bs @ Js
Jb

array([[-1.08420217e-19,  9.99999849e-01,  9.99999849e-01,
         9.99999849e-01, -1.20370622e-35],
       [ 1.11022302e-16,  5.49778687e-04,  5.49778687e-04,
         5.49778687e-04,  1.23259516e-32],
       [ 1.00000000e+00,  3.29407521e-36,  3.29407521e-36,
         3.29407521e-36,  1.00000000e+00],
       [-1.19998226e-01, -3.23269395e-05, -6.85034830e-05,
        -4.25528703e-05, -2.66449586e-17],
       [-6.59724770e-05,  5.87999051e-02,  1.24601907e-01,
         7.73999883e-02,  2.46201332e-37],
       [-5.68581742e-21,  1.19998244e-01,  6.87570958e-02,
        -2.66449627e-17,  2.21758445e-21]])

In [47]:
T_sb

array([[ 1.00000000e+00,  0.00000000e+00,  1.08420217e-19,
        -6.59724770e-05],
       [ 1.08420217e-19,  1.00000000e+00, -1.11022302e-16,
         1.19998226e-01],
       [-3.05188478e-20,  5.55111428e-17,  1.00000000e+00,
         6.65000860e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])